Settings

In [ ]:
use_half_precision_weight = True
hugging_face_token = ""

## Install dependencies

In [ ]:
!nvidia-smi

!pip install diffusers==0.2.4 
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

## Setup

In [ ]:
import os
import torch
from os import path
from PIL import Image
from torch import autocast
from google.colab import output
from diffusers import StableDiffusionPipeline
from IPython.display import Image,display_jpeg

output.enable_custom_widget_manager()

model_id = "CompVis/stable-diffusion-v1-4"
revision = "fp16"

if use_half_precision_weight:
  pipe = StableDiffusionPipeline.from_pretrained(model_id, revision=revision, torch_dtype=torch.float16, use_auth_token=hugging_face_token).to("cuda")
else:
  pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=hugging_face_token).to("cuda")


if not path.exists('outputs'):
    os.mkdir('outputs')


## Custom functions

In [ ]:
#pipe.safety_checker = lambda images, clip_input: (images, False)


def generate(prompt, pipe, seed, steps, width, height):  
  generator = torch.Generator("cuda").manual_seed(seed)

  if use_half_precision_weight:
    with autocast("cuda"):
      image = pipe(prompt, num_inference_steps=steps, height=height, width=width, generator=generator)["sample"][0]
  else: 
    image = pipe(prompt, num_inference_steps=steps, height=height, width=width, generator=generator)["sample"][0]

  return image

def save_image(image, used_prompt, used_seed):
  dir ="outputs/{}".format('_'.join(used_prompt.split(' ')))
  file_name = "{}.jpg".format(used_seed)
  saving_path = "{}/{}".format(dir, file_name)

  if not path.exists(dir):
    os.mkdir(dir)

  image.save(saving_path)

  return saving_path

## Run

In [ ]:
#@markdown With the same seed, the result will be the same
seed    = 40   #@param {type:"integer"}

#@markdown Image size. default: 512 x 512
width   = 512   #@param {type:"slider", min: 512, max: 1088, step: 64}
height  = 512   #@param {type:"slider", min: 512, max: 1088, step: 64}

#@markdown The more steps the better image (but takes more recource) default: 50
steps   = 50    #@param {type:"integer"}

#@markdown Number of images you want to generate
num     = 1     #@param {type:"integer"}

#@markdown Prompt
prompt = "a photograph of an astronaut riding a horse" #@param {type:"string"}


print("generating: \"{}\"".format(prompt))

for i in range(0, num):
  using_seed = seed + i
  print("seed: {}".format(using_seed))

  image = generate(prompt=[prompt], pipe=pipe, seed=using_seed, steps=steps, width=width, height=height)
  saved_path = save_image(image, prompt, using_seed)

  display_jpeg(Image(saved_path))


torch.cuda.empty_cache()